# Sample Code Demo

In [ ]:
import gc

import pickle
import torch
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.utils.data import DataLoader, random_split

from datautils import FixedFramesDataset, get_fixed_frame_data
from model_house import MLP, ResMLP
from trainutils import device, prediction, train


## Hyper Parameters

In [ ]:
nframes = 7
batch_size = 512
learning_rate = 1e-3
trial_name = "MLP-Demo"

## Define Dataset

In [ ]:
train_X, train_y = get_fixed_frame_data(
    nframes,
    feat_dir="./data/libriphone/feat/train",
    split_filepath="./data/libriphone/train_split.txt",
    labels_filepath="./data/libriphone/train_labels.txt",
)
test_X = get_fixed_frame_data(
    nframes, feat_dir="./data/libriphone/feat/test", split_filepath="./data/libriphone/test_split.txt"
)

train_X, test_X = train_X.numpy(), test_X.numpy()
# scaler = StandardScaler().fit(np.concatenate([train_X.reshape(-1, train_X.shape[-1]), test_X.reshape(-1, test_X.shape[-1])], axis=0))
with open("./scaler.skl", "rb") as fp:
    scaler = pickle.load(fp)
print("Scaling training data.")
train_X = scaler.transform(train_X.reshape(-1, train_X.shape[-1])).reshape(train_X.shape)
print("Scaling test data.")
test_X = scaler.transform(test_X.reshape(-1, test_X.shape[-1])).reshape(test_X.shape)


In [ ]:
train_X = torch.FloatTensor(train_X)
test_X = torch.FloatTensor(test_X)
train_dataset = FixedFramesDataset(train_X, train_y)
test_dataset = FixedFramesDataset(test_X)

del train_X, train_y, test_X
gc.collect()

train_len = int(len(train_dataset) * 0.8)
valid_len = len(train_dataset) - train_len
train_dataset, valid_dataset = random_split(train_dataset, [train_len, valid_len])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

## Define Model

In [ ]:
model = MLP(input_dim=(2 * nframes + 1) * 39, num_classes=41).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train(train_dataloader, valid_dataloader, model, criterion, optimizer, 50000, 100, trial_name)

## Prediction

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


model.load_state_dict(torch.load(f"./models/{trial_name}.ckpt"))
prediction(test_dataloader, model)
